In [11]:
#imports
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time

#set ups
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [12]:
url = 'https://cpj.org/data/?status=Killed&start_year=1992&end_year=2023&group_by=year&type%5B%5D=Journalist'
driver = webdriver.Chrome()
driver.get(url)

In [3]:
#select the dropdown menu you wish to make choices from
select = Select(driver.find_element(By.NAME, 'gender'))

#create your list of visible strings for genders
genders = ['Female', 'Male']

#create a list of numbers from 1 to 20
numbers = list(range(1,21))

#create an initially empty list
mydata = []

#loop through your genders
#select each one of those and do several other things for each gender in genders
for gender in genders:

    #select each one
    select.select_by_visible_text(gender)

    #sleep for a while
    time.sleep(5)

    driver.find_element(By.LINK_TEXT, 'Last').click()
    
    #get the page numbers that are visible while you are on the last page
    page_numbers = driver.find_elements(By.CLASS_NAME, 'page-item')
    #define the number of the last page of the table
    last_table_page = int(page_numbers[-3].text)

    #click on the First page to go back
    driver.find_element(By.LINK_TEXT, 'First').click()
    
    #define your currently active page 
    active_page = int(driver.find_element(By.CSS_SELECTOR, "li.page-item.active").text)
    
    #create a while loop!!!
    #we use while loops when we don't know the exact number of iterations
    #we just know that we want to keep looping through as long as a specific condition is true
    #we hereby wish to do several things while our active page is not the last one
    
    #so, while that happens...
    while active_page < last_table_page:
        
        #print your currently active page
        print(f"Page: {active_page}")
    
        #loop through your list of numbers
        for n in numbers:
            
            #create a dictionary called data
            data = {}
            
            #find each different name text
            name = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[1]/a').text
            organization = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[2]').text
            date = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[3]').text
            location = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[4]').text
            type_of_death = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[6]').text
            
            data['name'] = name
            data['gender'] = gender
            data['organization'] = organization
            data['date'] = date
            data['location'] = location
            data['type_of_death'] = type_of_death
            
            #check them out by printing them
            print(name, gender, organization, date, location, type_of_death)
            
            #append your initially empty list
            mydata.append(data)
            
        #find the "Next" button in your page
        button = driver.find_element(By.LINK_TEXT, "Next")  
        #click "Next"
        button.click()
        
        #redefine your active page
        active_page += 1
        
        #make your driver sleep for 5'' before it goes on
        time.sleep(5)
            
    else:
        print(f'Your currently active page is page number {active_page}. This should be the last page. I am getting the data and then I am done.')
        
        #get the data, store them, append your mydata list and that's it
        for n in numbers:
            
            data = {}
            
            #use try-except to avoid errors, especially when in your last page
            #because then most possible scenario is that your script will not find 20 names...
            try:
                name = driver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/div[2]/table/tbody/tr[{n}]/td[1]/a').text
            
                data['name'] = name
                data['gender'] = gender
                data['organization'] = organization
                data['date'] = date
                data['location'] = location
                data['type_of_death'] = type_of_death

                print(name, gender, organization, date, location, type_of_death)
                
            except:
                #if you fail finding data, just continue instead of breaking (throwing errors)
                continue
            
            mydata.append(data)

    #deselect your currently selected gender before you go on and start from the beginning again
    select.deselect_by_visible_text(gender)

Page: 1
Alaa Taher Al-Hassanat Female AlMajedat Media Network November 20, 2023 Israel and the Occupied Palestinian Territory Dangerous Assignment
Alexandra Tuttle Female The Wall Street Journal September 22, 1993 Georgia Crossfire
Alison Parker Female WDBJ7 August 26, 2015 USA Dangerous Assignment
Amparo Leonor Jiménez Pallares Female "QAP" and "En Vivo" August 11, 1998 Colombia Murder
Anastasiya Baburova Female Novaya Gazeta January 19, 2009 Russia Murder
Anja Niedringhaus Female The Associated Press April 4, 2014 Afghanistan Dangerous Assignment
Anna Politkovskaya Female Novaya Gazeta October 7, 2006 Russia Murder
Asiya Jeelani Female freelance April 20, 2004 India Dangerous Assignment
Atwar Bahjat Female Al-Arabiya February 23, 2006 Iraq Murder
Audrey Gaid Estrada Female 101.3 Grace Covenant FM March 17, 2022 Philippines Murder
Ayat Khadoura Female  November 20, 2023 Israel and the Occupied Palestinian Territory Dangerous Assignment
Ayelet Arnin Female Kan October 7, 2023 Israel an

In [4]:
df = pd.DataFrame(mydata)
df.shape

(1640, 6)

In [5]:
df

,name,gender,organization,date,location,type_of_death
0,Alaa Taher Al-Hassanat,Female,AlMajedat Media Network,"November 20, 2023",Israel and the Occupied Palestinian Territory,Dangerous Assignment
1,Alexandra Tuttle,Female,The Wall Street Journal,"September 22, 1993",Georgia,Crossfire
2,Alison Parker,Female,WDBJ7,"August 26, 2015",USA,Dangerous Assignment
3,Amparo Leonor Jiménez Pallares,Female,"""QAP"" and ""En Vivo""","August 11, 1998",Colombia,Murder
4,Anastasiya Baburova,Female,Novaya Gazeta,"January 19, 2009",Russia,Murder
5,Anja Niedringhaus,Female,The Associated Press,"April 4, 2014",Afghanistan,Dangerous Assignment
6,Anna Politkovskaya,Female,Novaya Gazeta,"October 7, 2006",Russia,Murder
7,Asiya Jeelani,Female,freelance,"April 20, 2004",India,Dangerous Assignment
8,Atwar Bahjat,Female,Al-Arabiya,"February 23, 2006",Iraq,Murder
9,Audrey Gaid Estrada,Female,101.3 Grace Covenant FM,"March 17, 2022",Philippines,Murder


In [6]:
df[df.gender=='Female'].shape

(125, 6)

In [7]:
df[df.gender=='Male'].shape

(1515, 6)

In [9]:
df.to_csv('CPJ_journalists_killed.csv',index=False)

In [10]:
driver.close()